In [1]:
#https://detectron2.readthedocs.io/
import detectron2
import logging
import torch
from detectron2.utils.logger import setup_logger
setup_logger()
logger = logging.getLogger('detectron2')

In [2]:
#Paths
from pathlib import Path
base_folder = Path('..')
data_folder = base_folder/'data'/'til2020'
train_imgs_folder = data_folder/'train'
train_annotations = data_folder/'train.json'
val_imgs_folder = data_folder/'val'
val_annotations = data_folder/'val.json'
test_imgs_folder = data_folder/'CV_interim_images'
test_annotations = data_folder/'CV_interim_evaluation.json'

#keep these the same
save_model_folder = base_folder/'ckpts'
load_model_folder = base_folder/'final_ckpts'

In [3]:
#register datasets. now they can be used as if they were native. remember to run fix_annotations.py to convert TIL2020 to proper COCO format
#to implement custom loaders, such as pickled, https://detectron2.readthedocs.io/modules/data.html?highlight=DatasetCatalog#detectron2.data.DatasetCatalog
from detectron2.data.datasets import register_coco_instances
register_coco_instances("til_train", {}, train_annotations, train_imgs_folder)
register_coco_instances("til_val", {}, val_annotations, val_imgs_folder)
register_coco_instances("til_test", {}, test_annotations, test_imgs_folder)

In [4]:
#https://detectron2.readthedocs.io/modules/config.html
#btw, i added some custom config options for my checkpointer and pipeline
from detectron2 import model_zoo
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
#cfg.SEED = 42

cfg.DATASETS.TRAIN = ("til_train",)
cfg.DATASETS.TEST = ("til_val",)
cfg.OUTPUT_DIR = str(save_model_folder)

cfg.DATALOADER.NUM_WORKERS = 12
cfg.DATALOADER.SAMPLER_TRAIN = "RepeatFactorTrainingSampler" #deals with class imbalance
cfg.DATALOADER.REPEAT_THRESHOLD = 0.5

cfg.SOLVER.IMS_PER_BATCH = 2 #batch_size
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.GAMMA = 0.5 #halves the learning rate at each milestone
cfg.SOLVER.STEPS = (10000000,)#(20000,60000,100000,140000) # milestones in iterations
#^I played with these but realised it had little effect: so keep BASE_LR at 0.000125*IMS_PER_BATCH

#Pipeline augmentation settings (i implemented these)
cfg.INPUT.CROP.ENABLED = False
cfg.INPUT.ENABLE_MINOR_AUGMENTS = True
cfg.INPUT.CROP.SIZE = [0.7, 0.9]
#cfg.INPUT.RAND_ROTATION = [-2,2]
cfg.INPUT.RAND_CONTRAST = [0.8,1.2]
cfg.INPUT.RAND_BRIGHTNESS = [0.8,1.2]
cfg.INPUT.RAND_SATURATION = [0.8,1.2]


cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 #number of categories
cfg.PRINT_EVERY = 200
cfg.SAVE_EVERY = 1000 #when using ValCheckpointer, it saves only if val loss is the minimum so far

cfg.SOLVER.MAX_ITER = -1
cfg.SOLVER.EPOCHS = 50 #facebook uses the paradigm of endless datastreams, as such epochs dont really exist
cfg.EPOCH_SIZE = int(8225/cfg.SOLVER.IMS_PER_BATCH)

logger.info(f"Loaded Config:\n{cfg.dump()}")

[06/16 18:50:07 detectron2]: Loaded Config:
CUDNN_BENCHMARK: false
DATALOADER:
  ASPECT_RATIO_GROUPING: true
  FILTER_EMPTY_ANNOTATIONS: true
  NUM_WORKERS: 12
  REPEAT_THRESHOLD: 0.5
  SAMPLER_TRAIN: RepeatFactorTrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: []
  PROPOSAL_FILES_TRAIN: []
  TEST:
  - til_val
  TRAIN:
  - til_train
EPOCH_SIZE: 4112
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: false
    SIZE:
    - 0.7
    - 0.9
    TYPE: relative_range
  ENABLE_MINOR_AUGMENTS: true
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN:
  - 640
  - 672
  - 704
  - 736
  - 768
  - 800
  MIN_SIZE_TRAIN_SAMPLING: choice
  RAND_BRIGHTNESS:
  - 0.8
  - 1.2
  RAND_CONTRAST:
  - 0.8
  - 1.2
  RAND_SATURATION:
  - 0.8
  - 1.2
MODEL:
  ANCHOR_GENERATOR:
    ANGLES:
    - - -90
      - 0
      - 90
    ASPECT_RATIOS:
    - - 0.5
      - 1.0
      - 2

In [5]:
from detectron2.modeling import build_model
from scripts.trainer import do_train,do_test
model = build_model(cfg)
logger.info(f"Created Model:\n{model}")

 (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
        (2): BottleneckBlock(
          (conv1): Conv2d(
            256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv2): Conv2d(
            64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
          )
          (conv3): Conv2d(
            64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
          )
        )
      )
      (res3): Sequential(
        (0): BottleneckBlock(
          (shortcut): Conv2d(
            256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False
            (norm): FrozenBatchNorm2d(num_features=512, eps=1e-05)
          )
          (conv1): Conv2d(
            256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False
            (n

In [6]:
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn-17999-best_val.pth")
do_train(cfg,model,"ft-til_resnet101_rcnn",resume=True)
#if this doesnt work correctly, check what model is set in the  file in /ckpts/last_checkpoint
#I uploaded the model file to the google drive
#have 40% of the images contain no 'person's to prevent overeagerness. figure out how to filter out 'person' images so you can delete the rest of the enormous coco dataset. 
#^use segmentation model, that way you can remove the background before passing it to the fashion model. How to indicate it is the background tho? put a checkerboard? somehow prevent person/clothes blending into background...

KeyboardInterrupt: 

In [6]:
#if do_train() hasnt been called (it loads the model) and you want to load the model
if True:
    from detectron2.checkpoint import DetectionCheckpointer
    cfg.MODEL.WEIGHTS = str(load_model_folder/"ft-til_resnet101_rcnn-17999-best_val.pth")
    DetectionCheckpointer(model, cfg.OUTPUT_DIR).resume_or_load(cfg.MODEL.WEIGHTS, resume=False)

evaluator = do_test(cfg,model)

[06/16 18:50:10 d2.data.datasets.coco]: Loaded 1474 images in COCO format from ../data/til2020/val.json
[06/16 18:50:10 d2.data.build]: Distribution of instances among all 5 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    tops    | 317          |  trousers  | 313          | outerwear  | 316          |
|  dresses   | 1338         |   skirts   | 174          |            |              |
|   total    | 2458         |            |              |            |              |
[06/16 18:50:10 d2.data.common]: Serializing 1474 elements to byte tensors and concatenating them all ...
[06/16 18:50:10 d2.data.common]: Serialized dataset takes 0.37 MiB
[06/16 18:50:10 d2.evaluation.evaluator]: Start inference on 1474 images
[06/16 18:50:12 d2.evaluation.evaluator]: Inference done 11/1474. 0.0821 s / img. ETA=0:02:00
[06/16 18:50:17 d2.evaluation.evaluator]: In

KeyboardInterrupt: 

In [7]:
evaluator = do_test(cfg,model,dataset_name='til_test')

[06/16 18:50:20 d2.data.datasets.coco]: Loaded 1474 images in COCO format from ../data/til2020/val.json
[06/16 18:50:20 d2.data.common]: Serializing 1474 elements to byte tensors and concatenating them all ...
[06/16 18:50:20 d2.data.common]: Serialized dataset takes 0.37 MiB
[06/16 18:50:20 d2.evaluation.evaluator]: Start inference on 1474 images
[06/16 18:50:22 d2.evaluation.evaluator]: Inference done 11/1474. 0.0838 s / img. ETA=0:02:03


KeyboardInterrupt: 

In [16]:
import cv2
from IPython.display import display
from PIL import Image
import numpy as np

from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
im = cv2.imread(train_imgs_folder/"69.jpg")
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
im_out = Image.fromarray(v.get_image()[:, :, ::-1]) #channels are reversed
#display(im_out)